In [25]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
from langchain.tools import tool
load_dotenv()

@dataclass
class Context:
    user_role: str

@wrap_model_call
def context_based_tools(
    request: ModelRequest,
    handler: Callable[[ModelRequest], ModelResponse]
) -> ModelResponse:
    """Filter tools based on Runtime Context permissions."""
    # Read from Runtime Context: get user role
    user_role = request.runtime.context.user_role

    if user_role == "admin":
        # Admins get all tools
        pass
    elif user_role == "editor":
        # Editors can't delete
        tools = [t for t in request.tools if t.name.startswith("cal_")]
        request = request.override(tools=tools)
    else:
        # Viewers get read-only tools
        tools = [t for t in request.tools if t.name.startswith("db_")]
        request = request.override(tools=tools)

    return handler(request)

llm = init_chat_model(
    model = "gpt-4o-mini",
    model_provider = "openai"
)

@tool
def cal_add()->str:
    """
    Adds
    """
    return "add"



@tool
def cal_sub()->str:
    """
    Sub
    """
    return "sub"


@tool
def cal_mul()->str:
    """
    Mul
    """
    return "mul"

@tool
def db_create()->str:
    """
    Creates table
    """
    return "created"
@tool
def db_read()->str:
    """
    Reads Table
    """
    return "Read"

@tool
def db_update()->str:
    """
    Updates Table
    """
    return "Updates"



agent = create_agent(
    model=llm,
    tools=[cal_add, cal_sub, cal_mul, db_create, db_read, db_update],
    middleware=[context_based_tools],
    context_schema=Context,
    system_prompt = """You are an helpful assistant. 
    You are allowed to work only with the tools that are available. 
    If the user query cannot be answered with the available tools respond with "Cannot be answered" """
)

In [26]:
agent.invoke(
    {"messages": [{"role": "user", "content": "1112+2223+7333?"}]},
    context=Context(user_role="db")
)

{'messages': [HumanMessage(content='1112+2223+7333?', additional_kwargs={}, response_metadata={}, id='d701ad71-84f2-4ddd-adaa-87c87b91ee01'),
  AIMessage(content='Cannot be answered.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 107, 'total_tokens': 112, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b547601dbd', 'id': 'chatcmpl-CgURkbZCex5A5VR14uy03e5HJtlAw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--5b56ce22-31a5-45de-9ba8-763142dbe45e-0', usage_metadata={'input_tokens': 107, 'output_tokens': 5, 'total_tokens': 112, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}